# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_691264b1ce20819187c25bf15157d855


### Upload Files

In [7]:
with open('tweet_textt.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-76R8SZS9V1dZXoTxFg52Q1


### Attach File to Vector Store

In [8]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-76R8SZS9V1dZXoTxFg52Q1


### Query the Vector Store

In [9]:
query = "the latest development in generativeAI"

In [10]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1e1
 Relevant score: 0.6712616619533297
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6022808670696832
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5964009774566718
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.5894568052546808
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.5601021939718683


## OpenAI Response API

### Simple Response

In [11]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [12]:
display(Markdown(simple_response.output_text))

As of the latest updates, generative AI continues to make significant strides across various domains:

1. **Large Language Models (LLMs):** Models like GPT-4 have become increasingly powerful, expanding capabilities in natural language understanding and generation. These models are being integrated into a wide range of applications, from chatbots to content creation tools.

2. **Multimodal Models:** AI systems capable of processing and generating text, images, audio, and video are becoming more sophisticated. OpenAI's DALL-E and similar models can generate detailed images from textual descriptions, while models like CLIP can understand and relate visual data with text.

3. **AI-Driven Creativity:** AI is being used to assist in creative fields such as music composition, art, and design. These tools are helping artists by generating novel ideas and automating tedious tasks.

4. **Ethics and Safety:** As generative models become more advanced, there is a growing focus on addressing ethical considerations, including bias, misinformation, and the potential for misuse. Researchers are developing guidelines and technologies to mitigate these issues.

5. **Personalization and Customization:** Businesses are using generative AI for personalized customer experiences, tailoring content and recommendations according to user preferences and behaviors.

6. **Research and Development:** Continuous advancements in architecture and training methods are optimizing models for better performance and efficiency, reducing computational requirements while improving output quality.

These developments are rapidly transforming how AI is applied across industries, enhancing its utility and impact.

### File Search Response

In [13]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [14]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include several exciting advancements:

1. **OpenAI's Sora2**: This tool allows users to create cinematic videos from a prompt, now with added features like audio, physics, and cameos, offering endless creativity for creators.

2. **Generative AI in Business**: AI is revolutionizing how businesses solve higher-value problems, with a growing demand for intelligent capabilities.

3. **AI in Creative Industries**: Generative AI is transforming creative industries by reshaping content creation, from code to designs and strategies.

4. **AI in Healthcare**: Generative AI is being used to support doctors with data interpretation and preliminary analysis, highlighting its potential in transforming healthcare.

These developments illustrate the broad impact of generative AI across various sectors, enhancing creativity, business innovation, and healthcare.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [15]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [16]:
display(Markdown(web_search_response.output_text))

Here is a structured, in-depth overview of the **latest developments in generative AI** as of Monday, November 10, 2025. This summary covers recent model launches, multimodal advances, improvements in efficiency, hardware trends, and real-world applications—each supported by citations from reliable sources.

---

## 1. Major Model Releases & Capabilities

- **OpenAI’s GPT‑5**, released on August 7, 2025, is a multimodal foundation model that combines reasoning and non-reasoning tasks under a unified interface. It powers tools like ChatGPT and Microsoft Copilot and offers state-of-the-art performance benchmarks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/GPT-5?utm_source=openai))

- **Google DeepMind’s “Nano Banana”** (officially Gemini 2.5 Flash Image), launched August 26, 2025, is a standout image-generation and editing tool within the Gemini ecosystem. It enables photorealistic edits, multi-image fusion, consistent subjects across edits, and SynthID watermarking. The tool went viral on social media, attracting over 10 million new users and enabling over 200 million image edits within weeks. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Nano_Banana?utm_source=openai))

- **Veo 3**, Google’s text-to-video model, released October 15, 2025, generates full videos with synchronized audio—speech, sound effects, and ambient noise—marking a leap beyond silent AI-generated video. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Veo_%28text-to-video_model%29?utm_source=openai))

- **Alibaba’s Qwen series** has made several notable launches throughout 2025:
  - In September, **Qwen3‑Max** and **Qwen3‑Next** debuted, with the latter featuring a hybrid attention mechanism, mixture-of-experts (MoE) structure, and multi-token prediction for faster, more efficient inference—especially in long contexts (>32K tokens) ([en.wikipedia.org](https://en.wikipedia.org/wiki/Qwen?utm_source=openai)).
  - **Qwen3‑Omni**, also released in September, is a mixed/multimodal model capable of processing text, images, audio, and video, delivering real-time streaming responses in text and natural speech. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Qwen?utm_source=openai))

- **Anthropic’s Claude 4 models**, specifically **Claude Opus 4** and **Sonnet 4**, launched in May 2025. These models introduce “extended thinking with tool use” (in beta), parallel tool execution, enhanced memory and instruction-following, and improved long-term contextual coherence. ([timesofindia.indiatimes.com](https://timesofindia.indiatimes.com/technology/artificial-intelligence/anthropic-launches-claude-opus-4-and-claude-sonnet-4-ai-models/articleshow/121355792.cms?utm_source=openai))

- **Claude 3.7 Sonnet**, released earlier in February 2025, brought “extended thinking mode”—a self-reflection phase before answering—to boost reasoning, factual accuracy, and creativity. ([reuters.com](https://www.reuters.com/technology/artificial-intelligence/anthropic-launches-advanced-ai-hybrid-reasoning-model-2025-02-24/?utm_source=openai))

---

## 2. Agentic and Multimodal Innovations

- **Perplexity AI’s Comet**, launched mid‑2025, is the first agentic browser capable of browsing across tabs, executing voice commands, and automating tasks (e.g., meeting booking). Initially available to Perplexity Max users, with broader access planned. ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai))

- **Hugging Face’s SmolLM3** is a 3B-parameter open-source model with up to 128K token context length. It outperforms equivalent-sized competitors and is optimized for efficiency and accessibility. ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai))

- **Liquid AI’s LFM2 models** (350M–1.2B parameters) are designed for edge deployment, offering 3× faster training and 2× faster inference speeds on CPUs, with a hybrid architecture for efficiency. ([linkedin.com](https://www.linkedin.com/pulse/top-generative-ai-updates-week-july-2-2025-kalyan-ks-oorrc?utm_source=openai))

---

## 3. Hardware & Infrastructure Supporting Generative AI

- **Nvidia’s new AI chips**, announced at GTC 2025 in March, include **Blackwell Ultra** (launching 2H 2025) and **Rubin AI** (late 2026), with **Rubin Ultra** slated for 2027. These chips are designed to accelerate generative and agentic AI workloads, reduce costs, and fuel future AI-driven infrastructure. Nvidia also demoed **Isaac GR00T N1** (for robotics) and introduced the **Newton physics engine** for simulation. ([apnews.com](https://apnews.com/article/457e9260aa2a34c1bbcc07c98b7a0555?utm_source=openai))

- **Qualcomm’s upcoming AI inference accelerators**, **AI200** and **AI250**, are targeted for 2026 and 2027, respectively. They feature advanced Hexagon NPUs, support Gen AI model encryption, micro-tile inferencing, and are optimized for AI workloads in data centers. ([tomshardware.com](https://www.tomshardware.com/tech-industry/artificial-intelligence/qualcomm-unveils-ai200-and-ai250-ai-inference-accelerators-hexagon-takes-on-amd-and-nvidia-in-the-booming-data-center-realm?utm_source=openai))

---

## 4. Broader Trends & Applications

- **Open-source models are gaining ground**, offering high performance comparable to proprietary models at lower cost—empowering startups and researchers. ([techinnoai.com](https://techinnoai.com/artificial-intelligence-machine-learning/generative-ai-in-2025-breakthroughs-real-world-applications/?utm_source=openai))

- **Model efficiency and cost reduction** are key development areas. New architectures enable high performance with reduced compute requirements, democratizing access to GenAI. ([techinnoai.com](https://techinnoai.com/artificial-intelligence-machine-learning/generative-ai-in-2025-breakthroughs-real-world-applications/?utm_source=openai))

- **AI and specialized hardware converge**, making generative AI faster and more energy-efficient, especially for edge and on-device deployment. ([techinnoai.com](https://techinnoai.com/artificial-intelligence-machine-learning/generative-ai-in-2025-breakthroughs-real-world-applications/?utm_source=openai))

- **Domain-specific innovations**:
  - Google DeepMind achieved an **85% improvement in protein prediction accuracy**, advancing genomics and drug discovery. ([konceptual.ai](https://konceptual.ai/trending/ai-breakthroughs-2025-genomics-robotics-revolution?utm_source=openai))
  - **Vision-Language-Action robotics** enables local, voice-controlled automation without cloud dependency—enhancing industrial and consumer applications. ([konceptual.ai](https://konceptual.ai/trending/ai-breakthroughs-2025-genomics-robotics-revolution?utm_source=openai))

- **AI governance and regulation** are developing globally. The EU AI Act has taken effect in 2025, and countries like the U.S., Japan, and Canada are implementing frameworks for ethical AI. ([ferip.com](https://ferip.com/5-powerful-breakthroughs-the-future-of-ai-in-2025/?utm_source=openai))

---

##  Summary: What to Watch Next

Generative AI in late 2025 features:

- Leading multimodal models like GPT‑5, Gemini’s Nano Banana, Veo 3, and Alibaba’s Qwen3‑Next & Omni.
- Agentic tools such as Claude’s extended thinking mode and Perplexity’s Comet browser.
- Efforts to optimize efficiency—from edge models to open-source frameworks.
- Major hardware accelerators from Nvidia and Qualcomm enhancing infrastructure agility.
- Real-world impact in domains like genomics and robotics, paired with accelerating regulation.

Let me know if you'd like deeper analysis on any of these developments—whether it's a specific model, hardware trend, or regulatory update.


### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [17]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here is a structured, in-depth overview of the **latest developments in generative AI** as of Monday

### Continue Query with Web Search

In [18]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [19]:
display(Markdown(continue_search_response.output_text))

Here are several **recent and diverse headlines** highlighting the latest developments in generative AI as of November 10, 2025. Each story is cited from reputable news sources, reflects different aspects of the field, and includes exact dates for clarity.

---

## OpenAI's "Sora" App Makes a Big Splash on Android
OpenAI's AI video creation app **Sora** debuted on Android on **November 6, 2025**, attracting nearly **470,000 downloads** in just one day—significantly outperforming the approximately **360,000 downloads** seen on iOS originally. This debut underscores the app’s surging popularity and the growing market for AI-powered video tools across mobile platforms. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Google Unveils DS STAR: AI That Writes Data Science Code End-to-End
Earlier this month (early November), **Google’s DS STAR** (Data Science Agent via Iterative Planning and Verification) was announced. This advanced framework employs multiple AI agents to interpret ambiguous business problems and convert them directly into executable Python code—without human intervention. DS STAR can process diverse data formats like CSV, JSON, Markdown, and unstructured text, pushing the envelope for automated data workflows. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Meta Expands “Vibes” AI Video Platform to Europe
Meta has extended its AI-generated short-video platform **Vibes** (similar to TikTok or Instagram Reels) to users in Europe. Each video on Vibes is fully generated by AI, providing a fresh content experience. This rollout marks a new phase in Meta’s integration of generative AI into social entertainment platforms. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Gemini 3 Pro Preview Spotted on Google’s Vertex AI
A **preview version** of **Gemini 3 Pro**—Google’s upcoming advanced AI model—has appeared on the **Vertex AI** platform under the identifier "gemini-3-pro-preview-11-2025." While not yet publicly available, this listing strongly suggests an imminent release. The model is expected to feature a **1 million-token context window**, significantly expanding capabilities for extended, context-rich interactions. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Gemini AI Adds “Deep Research,” Integrating Gmail and Drive
Google has introduced a **“Deep Research”** capability to its Gemini AI system. This feature enables the model to extract and synthesize information from a user’s Gmail, Google Drive, and Google Chat to generate comprehensive, researcher-level reports, showcasing deeper, more contextualized AI assistance across productivity tools. ([agiyes.com](https://www.agiyes.com/ainews/ai-news-from-november-1-7-2025/?utm_source=openai))

---

## Steam Sees Explosive Growth in Generative AI in Games
A recent study found that **one in five games released on Steam in 2025** incorporates generative AI—reflecting a **681% increase** year-on-year. Developers have disclosed GenAI usage in **7,818 titles**, which accounts for about **7% of Steam’s catalog**. Visual assets remain the most common use, followed by audio, narrative content, marketing, and coding. Notably, GenAI is also being used during gameplay for dynamic environments and content moderation. ([tomshardware.com](https://www.tomshardware.com/video-games/pc-gaming/1-in-5-steam-games-released-in-2025-use-generative-ai-up-nearly-700-percent-year-on-year-7-818-titles-disclose-genai-asset-usage-7-percent-of-entire-steam-library?utm_source=openai))

---

## Nvidia’s New Rubin AI Chips: Powering Future Generative and Agentic AI
At **GTC 2025** in March, Nvidia CEO Jensen Huang unveiled the next-generation AI chip strategies:

- **Blackwell Ultra** (launched 2nd half of 2025)
- **Rubin AI** (expected late 2026)
- **Rubin Ultra** (targeted for 2027)

These chips are designed to support complex generative and agentic AI workloads. Additionally, Nvidia introduced tools like the **Isaac GR00T N1** robotics model and the **Newton physics engine**—opening up new possibilities in AI-driven robotics, simulation, and autonomous systems. ([apnews.com](https://apnews.com/article/457e9260aa2a34c1bbcc07c98b7a0555?utm_source=openai))

---

## Google DeepMind Builds World-Modeling Team for Game and Robot Training
On **January 7, 2025**, Google DeepMind announced a new research team under **Tim Brooks** aimed at creating advanced “world models” capable of simulating physical environments. The models would support realistic training scenarios for robotics and interactive media—fueling progress toward more generalizable AI systems. The initiative aligns closely with existing projects like Veo video generation and the Gemini AI line. ([theverge.com](https://www.theverge.com/2025/1/7/24338053/google-deepmind-world-modeling-ai-team-gaming-robot-training?utm_source=openai))

---

### Summary Table (with Key Developments)

| Topic                         | Key Development & Date                |
|------------------------------|----------------------------------------|
| OpenAI Sora on Android       | 470K downloads on Nov 6, 2025          |
| Google DS STAR               | AI converting business problems to code|
| Meta Vibes                   | AI video platform expands to Europe    |
| Gemini 3 Pro Preview         | Appears on Vertex AI—soon release      |
| Gemini Deep Research         | Analyzes Gmail, Drive, Chat            |
| Steam Generative AI Usage    | 20% of 2025 Steam games include GenAI  |
| Nvidia Rubin AI Chips        | Blackwell Ultra (2025), Rubin AI (2026), Rubin Ultra (2027) |
| DeepMind World Modeling Team | Announced Jan 7, 2025                  |

---

Let me know if you'd like deeper insights into any of these stories—the technical innovations, their potential impact, or comparisons to competing developments across the generative AI landscape.

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [20]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [21]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several exciting advancements:

1. **OpenAI's Sora2**: This tool allows users to create cinematic videos from a simple prompt, incorporating audio, physics, and cameos. It's a game changer for creators, offering endless creative possibilities.

2. **Generative AI in Business**: AI is revolutionizing how businesses solve complex problems, with a focus on higher-value tasks. The demand for intelligent capabilities is growing, positioning AI as a central component in innovation.

3. **AI in Creative Industries**: Generative AI is transforming creative fields by enabling the creation of content, designs, and strategies, reshaping how businesses innovate.

4. **AI in Healthcare**: Generative AI is being used to support doctors with data interpretation and preliminary analysis, highlighting its potential to transform healthcare.

5. **AI in Marketing**: AI is making significant impacts in marketing, such as creating cinematic food commercials quickly and efficiently without the need for traditional production setups.

These developments illustrate the broad and transformative impact of generative AI across various sectors.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [30]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_691269f837d48191830cac7760f3c8f8


In [31]:
with open('news2.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-CeQEh5Nm6EzAYroqzAMsTy


In [32]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-CeQEh5Nm6EzAYroqzAMsTy


In [33]:
query="Who is paying for new fighter jets"

In [34]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

Goal: More Jets, But Who’s Paying?
Nov. 7, 2025 | By John A. Tirpak


Share Article
The Air Force’s 
 Relevant score: 0.03278688524590164
Retired Lt. Gen. David Deptula, dean of AFA’s Mitchell Institute for Aerospace Studies, called the h
 Relevant score: 0.03225806451612903
Jeremiah “J.J.” Gertler, director of The Defense Concepts Organization, noted that Lockheed Martin’s
 Relevant score: 0.03149801587301587
A separate, Pentagon-funded, Congressionally directed study was done in parallel to “The Force We Ne
 Relevant score: 0.03149801587301587


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [35]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [36]:
display(Markdown(file_search_response.output_text))

The funding for new fighter jets is a complex issue. The Air Force's plan to increase its fighter capacity involves significant costs, estimated at more than $15 billion per year. This funding would need to be sourced from within the Air Force's existing budget, potentially requiring cuts from other areas. The document suggests that reallocating funds from other services, such as the Army, might be a possibility.

In [37]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [38]:
display(Markdown(web_search_response.output_text))

The procurement of new fighter jets is typically funded by national governments, often as part of their defense budgets. The specific financial arrangements can vary depending on the country, the defense contractors involved, and any special financing or cost-sharing agreements that may be in place. In some cases, purchases might be supported through multi-national partnerships or collaborations. For details on a particular country's fighter jet acquisition, checking their defense department announcements or budget documents would provide the most accurate information.

In [52]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Using my uploaded notes and the latest web information, summarize the current trends on this topic",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [53]:
display(Markdown(combined_search_response.output_text))

The current trends regarding the Air Force's new fighter goals focus on increasing the number of fighter jets by 30% over the next decade. This plan aims to revitalize the legacy fighter fleet and procure new models like the F-15EX, F-35A, and F-47. The goal is to reach a total of 1,558 manned combat-coded aircraft, which would support 65 fighter squadrons.

However, achieving this goal presents challenges, particularly in terms of funding and production capacity. The Air Force would need to significantly increase its annual purchases of F-35s and F-15EXs, which is currently beyond its procurement capabilities and the production capacity of manufacturers.

The plan also includes the development of Collaborative Combat Aircraft (CCAs), which are uncrewed, semi-autonomous fighter escorts. These would be additional to the crewed fighter numbers.

Critics have pointed out that the plan lacks a clear force-sizing rationale and does not adequately address the diverse potential war scenarios beyond the focus on China and Taiwan. The financial aspect is also a concern, as the Air Force would need to reallocate funds from other areas to meet the increased budget requirements.